In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install emoji unidecode

     |████████████████████████████████| 51kB 2.9MB/s 
     |████████████████████████████████| 245kB 7.5MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=6801b10f5b33a4eceaa2fa31d76f238f55c549cedf92be75fd17a880ecf1d07d
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [13]:
import nltk
nltk.download('punkt')
from nltk.tokenize import MWETokenizer, word_tokenize, RegexpTokenizer
import re
import nltk
import unicodedata

multiple_punctuation_pattern = re.compile(r"([\"\.\?\!\,\:\;\-])(?:[\"\.\?\!\,\:\;\-]){1,}")
word_tokenizer = MWETokenizer(separator='')
multiple_emoji_pattern = re.compile(u"(["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\u00a9"
        u"\u00ae"
        u"\u2000-\u3300"
        "]){1,}", flags= re.UNICODE )

normalizer = {'òa': 'oà',
              'óa': 'oá',
              'ỏa': 'oả',
              'õa': 'oã',
              'ọa': 'oạ',
              'òe': 'oè',
              'óe': 'oé',
              'ỏe': 'oẻ',
              'õe': 'oẽ',
              'ọe': 'oẹ',
              'ùy': 'uỳ',
              'úy': 'uý',
              'ủy': 'uỷ',
              'ũy': 'uỹ',
              'ụy': 'uỵ',
              'Ủy': 'Uỷ'}
correct_mapping = {
      "m": "mình",
      "mik": "mình",
      "ko": "không",
      "k": " không ",
      "kh": "không",
      "khong": "không",
      "kg": "không",
      "khg": "không",
      "tl": "trả lời",
      "r": "rồi",
      "ok": "tốt",
      "dc": "được",
      "vs": "với",
      "đt": "điện thoại",
      "thjk": "thích",
      "thik": "thích",
      "qá": "quá",
      "trể": "trễ",
      "bgjo": "bao giờ",
      "''": '"',
      "``": '"'
}

def normalize_text(text):
  for absurd, normal in normalizer.items():
    text = text.replace(absurd, normal)

  # for l in vn_location:
  #   text = text.replace(l, ' location ')

  return text

def tokmap(tok):
  if tok.lower() in correct_mapping:
      return correct_mapping[tok.lower()]
  else:
      return tok

def preprocess(text):
  global i
  text = multiple_emoji_pattern.sub(r"\g<1> ", text) # \g<1>
  text = multiple_punctuation_pattern.sub(r" \g<1> ", text)
  text = unicodedata.normalize("NFC", text)
  text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b(\/)?', 'url', text)
  text = re.sub("\.", " . ", text)
  text = re.sub("'", "' ", text)
  text = re.sub('"', '" ', text)
  text = re.sub('/', ' / ', text)
  text = re.sub('-', ' - ', text)
  text = re.sub(',', ' , ', text)
  text = re.sub(r'\s{2,}', ' ', text)
  text = normalize_text(text)
  # text = re.sub(r'\#[^\s]+', ' hastag ', text)
  text = re.sub(r'(|\s)([\d]+k)(\s|$)', ' cureency_k ', text)
  text = re.sub(r'(([\d]{2,4}\s){2,}([\d]+)?|(09|01|[2|6|8|9]|03)+([0-9]{8})\b)', ' phone_number ', text)
  text = re.sub(r'\d', "_digit", text)
  tokens = word_tokenizer.tokenize(word_tokenize(text))
  tokens = list(map(tokmap, tokens))
  # return tokens
  return ' '.join(tokens)

print(preprocess('Thầy Trần Mai Vũ à một cổ động viên trung thành của đội bóng Manchester United'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Thầy Trần Mai Vũ à một cổ động viên trung thành của đội bóng Manchester United


In [ ]:
with open('/content/drive/MyDrive/KPDL_1/sentiment_analysis_test_unlabel.v1.0.txt') as f:
    sentiment_analysis_test = f.read().strip().split('\n')
    sentiment_analysis_test = [ preprocess(e) for e in sentiment_analysis_test]
print(sentiment_analysis_test[:10])
print(len(sentiment_analysis_test))

['\ufeffKhách sạn tiêu chuẩn , nhưng phục vụ kém . Điểm hình tổ chức tiệc buffet đêm giao thừa , lượng người không quá đông , nhưng khâu chuẩn bị lủng củng , nhân viên chưa niềm nở với khách , thức ăn không được tươi ngon , phục vụ không đúng như thực đơn buffet ( nói tổng thể là khách sạn tiêu chuẩn phục vụ không chuẩn ) . Phục vụ và ăn uống', 'Khách sạn nằm ở trung tâm , tiện đi lại , chỉ _digitp đến mặt đường Trần Phú . Nhân viên phục vụ nhiệt tình thân thiện , tiện nghi khách sạn bình thường , mới hoạt động nên còn thiếu nhiều thiết bị , dịch vụ . Đánh giá khách sạn', 'Không có gì phải phàn nàn . Đánh giá', 'Phòng dơ , nhà tắm dơ , lúc mình tới là t_digit muốn upgrade phòng nhưng khách sạn nói là không còn phòng nữa . Không thích lắm chất lượng phòng ở đó , chỉ có vị trí gần biển ! Lần sau sẽ không ghé khách sạn này nữa ! . danh gia khach san victory', 'Khách sạn phục vụ chu đáo . Dịch vụ tốt . Nhân viên nhiệt tình . Đồ ăn hợp khẩu vị . Khách sạn Victory', 'Ấn tượng đầu tiên có lẽ 

In [ ]:
with open('sentiment_analysis_fasttext_test.txt', 'w') as f:
  f.write('\n'.join(sentiment_analysis_test))

In [ ]:
with open('/content/drive/MyDrive/KPDL_1/sentiment_analysis_train_v1.0.txt') as f:
  sentiment_analysis_train = f.read().strip().split('\n')
  sentiment_analysis_train = [ line.split(' ',1) for line in sentiment_analysis_train]
  sentiment_analysis_train = [ [lables, preprocess(descriptions)] for lables, descriptions in sentiment_analysis_train]
  sentiment_analysis_train = [ ' '.join(e) for e in sentiment_analysis_train]
print(sentiment_analysis_train[:2])
with open('sentiment_analysis_fasttext_train.txt', 'w') as f:
  f.write('\n'.join(sentiment_analysis_train))

['__label__tot Good ratio price / service . Good advices for the national park and village . Tốt', '__label__trung_binh Trang thiết bị vệ sinh hơi cũ . Vệ sinh sạch sẽ , nhân viên thân thiện , địa điểm tốt . Một khách sạn sạch sẽ , thoải mái , dễ chịu , địa điểm tốt .']


In [ ]:
!pip install fasttext

     |████████████████████████████████| 71kB 3.4MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3043060 sha256=465d0876644a38ef20ae01e67062a1bfbda07b8c3481678502827094d50bb2f0
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
import fasttext

#Train the model
model = fasttext.train_supervised(input="sentiment_analysis_fasttext_train.txt", lr=0.22, wordNgrams=3) #  epoch=5)# dim=10)
# model.save_model("sell_dectection_60K_fasttext.bin")
prediction = []

for i in range(len(sentiment_analysis_test)):
  prediction.append(model.predict(sentiment_analysis_test[i])[0][0] )
print(len(prediction))


20241


In [ ]:
with open('sentiment_analysis_result.txt', 'w') as f:
  f.write('\n'.join(prediction))